# Topic Modeling with Gensim

This notebook explores two topic modeling algorithms with the Gensim package:

1. LDA - Latent Dirichlet Allocation, an algorithm using probabilistic graphical models
2. LSI - Latent Semantic Indexing, also called LSA, Latent Semantic Analysis, an algorithm using Singular Value Decomposition (SVD), a dimensionality reduction technique 

### Gensim

The gensim Python library can be installed with pip/pip3. Documentation about the gensim API as well as tutorials can be found on the [web site](https://radimrehurek.com/gensim/index.html)


### A toy corpus

A very small corpus will be used to illustrate the steps involved in running the algorithms with Gensim. In fact, the corpus is so small that it is unlikely that good results will be achieved. The corpus consists of only 4 documents, each representing the text of one section of a textbook. The code block below reads in the anatomy, business law, economics, and geography texts, creating a list of docs.

In [8]:
import re

num_docs = 4
docs = []

with open('../school_texts/anat.txt', 'r') as f:
    doc_anat = f.read().lower()
    doc_anat = doc_anat.replace('\n', ' ')
    docs.append(doc_anat)

with open('../school_texts/buslaw.txt', 'r') as f:
    doc_buslaw = f.read().lower()
    doc_buslaw = doc_buslaw.replace('\n', ' ')
    docs.append(doc_buslaw)
    
with open('../school_texts/econ.txt', 'r') as f:
    doc_econ = f.read().lower()
    doc_econ = doc_econ.replace('\n', ' ')
    docs.append(doc_econ)
    
with open('../school_texts/geog.txt', 'r') as f:
    doc_geog = f.read().lower()
    doc_geog = doc_geog.replace('\n', ' ')
    docs.append(doc_geog)
    
# look at part of each document
for i in range(num_docs):
    print(docs[i][:50])

the autonomic nervous system the autonomic nervous
16.1 theory of contract remedies purpose of remedi
22 | inflation inflation is a general and ongoing 
chapter 13 the pacific and antarctica the immense 


In [9]:
# gensim and nltk imports
from gensim import models, corpora
from nltk import word_tokenize
from nltk.corpus import stopwords

In [10]:
NUM_TOPICS = 8

In [88]:
# preprocess docs
def preprocess(docs, stopwords):
    """
    Tokenize, remove stopwords and non-alpha tokens.
    param: docs - a list of raw text documents
    return: a list of processed tokens
    """
    
    processed_docs = []
    for doc in docs:
        tokens = [t for t in word_tokenize(doc.lower()) if t not in stopwords
                 and t.isalpha()]
        processed_docs.append(tokens)
        
    return processed_docs


In [90]:
preprocessed_docs = preprocess(docs, stopwords.words('english'))

In [91]:
for i in range(num_docs):
    print(preprocessed_docs[i][:5])

['autonomic', 'nervous', 'system', 'autonomic', 'nervous']
['theory', 'contract', 'remedies', 'purpose', 'remedies']
['inflation', 'inflation', 'general', 'ongoing', 'rise']
['chapter', 'pacific', 'antarctica', 'immense', 'tropical']


In [92]:
# the dictionary maps words to id numbers
dictionary = corpora.Dictionary(preprocessed_docs)

In [93]:
print('len of dictionary:', len(dictionary))
print('some items:', dictionary[0], dictionary[4053])

len of dictionary: 4054
some items: abdominal zone


In [94]:
# represent the doc tokens in numeric form
corpus = [dictionary.doc2bow(tokens) for tokens in preprocessed_docs]

In [54]:
# each doc in the corpus is now a bag of words
# printing the first few 'words' in the bag of words confirms that word order is lost
print(corpus[0][:5])
print(dictionary[4], dictionary[2], dictionary[1])

[(0, 4), (1, 2), (2, 1), (3, 1), (4, 1)]
accelerator absorbed ability


In [109]:
# build an LDA model
lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

In [61]:
print("LDA Model Results")
for i in range(NUM_TOPICS):
    print("\nTopic #%s:" % i, lda_model.print_topic(i, 10))

# the results below show for each topic, the top 10 words associated with that topic, along with the importance factor

LDA Model Results

Topic #0: 0.010*"system" + 0.008*"party" + 0.008*"damages" + 0.006*"inflation" + 0.006*"contract" + 0.005*"islands" + 0.005*"sympathetic" + 0.005*"one" + 0.005*"autonomic" + 0.005*"may"

Topic #1: 0.014*"inflation" + 0.007*"price" + 0.006*"would" + 0.006*"islands" + 0.005*"rate" + 0.005*"goods" + 0.005*"contract" + 0.005*"one" + 0.005*"prices" + 0.004*"party"

Topic #2: 0.009*"islands" + 0.008*"inflation" + 0.006*"system" + 0.005*"island" + 0.004*"price" + 0.004*"one" + 0.004*"new" + 0.004*"would" + 0.004*"sympathetic" + 0.003*"rate"

Topic #3: 0.018*"inflation" + 0.007*"contract" + 0.007*"goods" + 0.007*"price" + 0.006*"damages" + 0.006*"party" + 0.006*"would" + 0.005*"may" + 0.005*"one" + 0.005*"rate"

Topic #4: 0.012*"inflation" + 0.007*"system" + 0.006*"islands" + 0.005*"one" + 0.005*"would" + 0.005*"price" + 0.004*"prices" + 0.004*"sympathetic" + 0.004*"many" + 0.004*"goods"

Topic #5: 0.006*"system" + 0.006*"islands" + 0.005*"contract" + 0.005*"damages" + 0.005

In [74]:
for i in range(NUM_TOPICS):
    top_words = [t[0] for t in lda_model.show_topic(i, 9)]
    print("\nTopic", str(i), ':', top_words)


Topic 0 : ['system', 'party', 'damages', 'inflation', 'contract', 'islands', 'sympathetic', 'one', 'autonomic']

Topic 1 : ['inflation', 'price', 'would', 'islands', 'rate', 'goods', 'contract', 'one', 'prices']

Topic 2 : ['islands', 'inflation', 'system', 'island', 'price', 'one', 'new', 'would', 'sympathetic']

Topic 3 : ['inflation', 'contract', 'goods', 'price', 'damages', 'party', 'would', 'may', 'one']

Topic 4 : ['inflation', 'system', 'islands', 'one', 'would', 'price', 'prices', 'sympathetic', 'many']

Topic 5 : ['system', 'islands', 'contract', 'damages', 'inflation', 'party', 'autonomic', 'one', 'sympathetic']

Topic 6 : ['inflation', 'goods', 'system', 'would', 'price', 'prices', 'sympathetic', 'may', 'rate']

Topic 7 : ['system', 'sympathetic', 'inflation', 'damages', 'one', 'contract', 'islands', 'autonomic', 'price']


In [69]:
# look at weights for top 10 words in topic 0
lda_model.show_topic(0, 10)

[('system', 0.009830139),
 ('party', 0.007665353),
 ('damages', 0.007519758),
 ('inflation', 0.0063338433),
 ('contract', 0.0059979996),
 ('islands', 0.005396935),
 ('sympathetic', 0.005083735),
 ('one', 0.0049147718),
 ('autonomic', 0.0048522805),
 ('may', 0.0047706696)]

In [107]:
print("LDA Model 1 Perplexity:", lda_model.log_perplexity(corpus))

from gensim.models.coherencemodel import CoherenceModel

coherence1 = CoherenceModel(model=lda_model,
                           texts=preprocessed_docs, dictionary=dictionary, coherence='c_v')
print('Coherence score:', coherence1.get_coherence())

LDA Model 1 Perplexity: -10.880851592753286
Coherence score: 0.3654794360221424


## Visualization

The pyLDAvis package enables visualization of topics and documents. The package can be installed with pip or pip3.

In [78]:
import pyLDAvis
from pyLDAvis import gensim
pyLDAvis.enable_notebook()

In [83]:
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)

vis

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.025444  0.010595       1        1  42.089947
0      0.012582  0.012439       2        1  31.978855
2      0.008166 -0.013427       3        1  25.887436
1     -0.026179 -0.008362       4        1   0.023404
5      0.015646 -0.001818       5        1   0.011321
6     -0.000109  0.002979       6        1   0.003013
4      0.001455 -0.005311       7        1   0.003013
7      0.013882  0.002905       8        1   0.003013, topic_info=     Category        Freq             Term       Total  loglift  logprob
2746  Default  196.000000        inflation  196.000000  30.0000  30.0000
1128  Default   96.000000           system   96.000000  29.0000  29.0000
3580  Default   95.000000          islands   95.000000  28.0000  28.0000
289   Default   92.000000         contract   92.000000  27.0000  27.0000
1982  Default   77.000000            price   77.000000  26.0000  26.0000
1245  Default   78.000000            would   78.000000  25.0000  25.0000
776   Default   79.000000              one   79.000000  24.0000  24.0000
1472  Default   92.000000          damages   92.000000  23.0000  23.0000
1682  Default   76.000000            goods   76.000000  22.0000  22.0000
1928  Default   92.000000            party   92.000000  21.0000  21.0000
905   Default   68.000000             rate   68.000000  20.0000  20.0000
680   Default   70.000000              may   70.000000  19.0000  19.0000
1119  Default   50.000000      sympathetic   50.000000  18.0000  18.0000
2910  Default   54.000000           prices   54.000000  17.0000  17.0000
678   Default   53.000000             many   53.000000  16.0000  16.0000
112   Default   45.000000        autonomic   45.000000  15.0000  15.0000
3578  Default   47.000000           island   47.000000  14.0000  14.0000
1181  Default   43.000000              two   43.000000  13.0000  13.0000
1743  Default   48.000000         interest   48.000000  12.0000  12.0000
1868  Default   46.000000              new   46.000000  11.0000  11.0000
57    Default   45.000000             also   45.000000  10.0000  10.0000
2736  Default   40.000000            index   40.000000   9.0000   9.0000
2886  Default   39.000000           people   39.000000   8.0000   8.0000
800   Default   35.000000  parasympathetic   35.000000   7.0000   7.0000
1871  Default   44.000000     nonbreaching   44.000000   6.0000   6.0000
470   Default   34.000000           fibers   34.000000   5.0000   5.0000
1345  Default   54.000000           breach   54.000000   4.0000   4.0000
430   Default   36.000000          example   36.000000   3.0000   3.0000
2327  Default   39.000000             year   39.000000   2.0000   2.0000
1244  Default   36.000000            world   36.000000   1.0000   1.0000
...       ...         ...              ...         ...      ...      ...
1222   Topic8    0.000685         visceral   13.039762   0.5557  -6.5955
919    Topic8    0.001297        receptors   27.954828   0.4318  -5.9568
470    Topic8    0.001540           fibers   34.646290   0.3886  -5.7854
112    Topic8    0.001881        autonomic   45.782696   0.3100  -5.5853
354    Topic8    0.000647        digestive   12.303102   0.5574  -6.6520
800    Topic8    0.001519  parasympathetic   35.713364   0.3450  -5.7987
749    Topic8    0.001129           neuron   24.909760   0.4083  -6.0956
381    Topic8    0.000996            drugs   21.814262   0.4158  -6.2208
496    Topic8    0.001042          ganglia   23.911388   0.3694  -6.1755
1181   Topic8    0.001509              two   43.620846   0.1380  -5.8057
625    Topic8    0.000954            large   22.823574   0.3272  -6.2642
776    Topic8    0.002137              one   79.218307  -0.1104  -5.4574
931    Topic8    0.000950           region   23.216707   0.3060  -6.2683
1472   Topic8    0.002188          damages   92.098412  -0.2377  -5.4341
473    Topic8    0.000869           figure   20.356531   0.

## Interpreting the visualization

Each bubble in the left part of the visualization represents a topic. The larger the bubble, the more prevalent that topic is in the corpus. A good topic model will have well-separated, not overlapping bubbles, not all in one quadrant. The closer bubbles are, the more similar their topics. Overlapping bubbles can be an indication that there are too many topics. 

Moving the cursor over a bubble, important words in that topic are highlighted in red on the right side of the visualization. The red bars indicate term frequency of each word in the topic.  

Visualizing the topic models in this way helps identify problems in the topic model:

* topics include unimportant words such as 'would', 'one', 'may', 'many', 'also'; using an expanded set of stop words would help alleviate some of this
* words and their plurals show up in topics, see 'island' and 'islands'
* the same words appear in multiple topics; for example 'island' and 'islands'; this is often an indication that the number of topics is too large

Documentation for the visualization program is [available here](https://pyldavis.readthedocs.io/en/latest/modules/API.html)

## Run LDA again

Another LDA model is built in the next code block, with the following changes:

* additional words were added to the standard stopwords
* the number of topics is changed to 4


In [98]:
enhanced_stopwords = stopwords.words('english') 
enhanced_stopwords += ['could', 'may', 'would', 'many', 'also']
preprocessed_docs2 = preprocess(docs, enhanced_stopwords)

# the dictionary maps words to id numbers
dictionary2 = corpora.Dictionary(preprocessed_docs2)

# represent the doc tokens in numeric form
corpus2 = [dictionary2.doc2bow(tokens) for tokens in preprocessed_docs2]

# the dictionary maps words to id numbers
dictionary2 = corpora.Dictionary(preprocessed_docs2)

# build another LDA model, this time with 4 topics
lda_model2 = models.LdaModel(corpus=corpus2, num_topics=4, id2word=dictionary2)
vis2 = pyLDAvis.gensim.prepare(lda_model2, corpus2, dictionary2)
vis2

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.009174 -0.001647       1        1  46.690643
0     -0.007248  0.002302       2        1  28.085608
3      0.003118  0.003489       3        1  20.435747
1     -0.005045 -0.004144       4        1   4.788004, topic_info=     Category        Freq          Term       Total  loglift  logprob
2741  Default  143.000000     inflation  143.000000  30.0000  30.0000
1977  Default   61.000000         price   61.000000  29.0000  29.0000
1677  Default   57.000000         goods   57.000000  28.0000  28.0000
3575  Default   96.000000       islands   96.000000  27.0000  27.0000
2905  Default   46.000000        prices   46.000000  26.0000  26.0000
1125  Default  101.000000        system  101.000000  25.0000  25.0000
902   Default   58.000000          rate   58.000000  24.0000  24.0000
2731  Default   31.000000         index   31.000000  23.0000  23.0000
288   Default   84.000000      contract   84.000000  22.0000  22.0000
773   Default   78.000000           one   78.000000  21.0000  21.0000
1923  Default   79.000000         party   79.000000  20.0000  20.0000
1738  Default   36.000000      interest   36.000000  19.0000  19.0000
2322  Default   28.000000          year   28.000000  18.0000  18.0000
2881  Default   40.000000        people   40.000000  17.0000  17.0000
1178  Default   44.000000           two   44.000000  16.0000  16.0000
977   Default   24.000000          rise   24.000000  15.0000  15.0000
637   Default   28.000000         level   28.000000  14.0000  14.0000
1467  Default   82.000000       damages   82.000000  13.0000  13.0000
1863  Default   43.000000           new   43.000000  12.0000  12.0000
1156  Default   26.000000          time   26.000000  11.0000  11.0000
1241  Default   32.000000         world   32.000000  10.0000  10.0000
429   Default   36.000000       example   36.000000   9.0000   9.0000
3573  Default   51.000000        island   51.000000   8.0000   8.0000
3189  Default   32.000000    antarctica   32.000000   7.0000   7.0000
1116  Default   65.000000   sympathetic   65.000000   6.0000   6.0000
1866  Default   39.000000  nonbreaching   39.000000   5.0000   5.0000
1341  Default   39.000000        breach   39.000000   4.0000   4.0000
737   Default   31.000000       nervous   31.000000   3.0000   3.0000
2386  Default   19.000000        basket   19.000000   2.0000   2.0000
1083  Default   27.000000        states   27.000000   1.0000   1.0000
...       ...         ...           ...         ...      ...      ...
637    Topic4    1.926886         level   28.441561   0.3471  -6.0051
2138   Topic4    1.216739      services   17.746008   0.3591  -6.4649
868    Topic4    0.836790       problem   11.697148   0.4015  -6.8392
2905   Topic4    2.806434        prices   46.898796   0.2230  -5.6291
2940   Topic4    0.996464         rates   14.328238   0.3733  -6.6646
212    Topic4    1.197397       changes   17.757511   0.3424  -6.4809
1156   Topic4    1.701941          time   26.944605   0.2770  -6.1293
1241   Topic4    1.912058         world   32.647736   0.2015  -6.0128
902    Topic4    3.057372          rate   58.475609   0.0880  -5.5435
1633   Topic4    1.219325         fixed   18.751690   0.3061  -6.4627
1692   Topic4    1.316204        higher   20.651642   0.2860  -6.3863
713    Topic4    1.495119          much   24.331659   0.2495  -6.2588
1083   Topic4    1.619894        states   27.358736   0.2124  -6.1787
3189   Topic4    1.830116    antarctica   32.165432   0.1725  -6.0566
2881   Topic4    2.172112        people   40.604084   0.1109  -5.8853
3575   Topic4    4.021967       islands   96.263252  -0.1363  -5.2693
773    Topic4    3.321224           one   78.981400  -0.1298  -5.4607
1178   Topic4    2.229404           two   44.886814   0.0366  -5.8593
1125   Topic4    3.842006        system  101.598122  -0.2360  -5.3150
288    Topic4    3.379742      contract   84.937439  -0.1851  -

## Comparing the two models

We know from this small corpus that there really are four topics:

1. the autonomic nervous system
2. theory of contract remedies
3. the Pacific and Antarctica
4. inflation

There are several problems revealed by visual inspection of the topics.
* The topic identified as '1' above is a mix of key words from all 4 documents. 
* The topic identifed as '2' above seems closest to the inflation topic with key words like: inflation, system, rate, price, goods, and damages.
* The topic identifed as '3' above mixes key words: inflation, system, islands, party, damages, sympathetic. These seem to be key words from all 4 documents.
* The topic identified as '4' seems to be a weak topic with low estimated frequencies.

These results are not impressive. Only topic '2' seems to have honed in on a topic that is consistent with the documents. 

In [108]:
print("LDA Model 2 Perplexity:", lda_model2.log_perplexity(corpus2))

coherence2 = CoherenceModel(model=lda_model2,
                           texts=preprocessed_docs2, dictionary=dictionary2, coherence='c_v')
print('Coherence score:', coherence2.get_coherence())

LDA Model 2 Perplexity: -8.303100841208572
Coherence score: 0.428920836931636


## Conclusion

Overall, LDA gave interest results but the resuls don't conform to human intuition about the corpus, and the scores were not high. The main problem with applying LDA to this tiny corpus is the fact that the corpus is tiny. There are not enough words in enough contexts for the algorithm to learn very much. In fact, it's surprising that it learned as much as it did. 

The purpose of demonstrating LDA on this small corpus is to show the steps involved in a notebook that will run quickly. A later project notebook will show results from running LDA on a larger corpus.

## LSI

In [59]:
# build an LSI model
lsi_model = models.LsiModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

In [60]:
print("LSI Model Results")
for i in range(NUM_TOPICS):
    print("\nTopic #%s:" % i, lsi_model.print_topic(i, 10))

LSI Model Results

Topic #0: 0.628*"inflation" + 0.233*"price" + 0.219*"goods" + 0.191*"prices" + 0.185*"rate" + 0.136*"index" + 0.106*"would" + 0.106*"one" + 0.105*"year" + 0.103*"interest"

Topic #1: -0.440*"system" + -0.272*"sympathetic" + 0.225*"inflation" + -0.210*"autonomic" + -0.178*"parasympathetic" + -0.171*"fibers" + -0.149*"receptors" + -0.142*"nervous" + -0.123*"postganglionic" + -0.123*"ganglia"

Topic #2: -0.350*"party" + -0.350*"damages" + -0.330*"contract" + 0.218*"inflation" + -0.186*"islands" + -0.181*"breach" + -0.177*"nonbreaching" + -0.172*"would" + -0.154*"may" + -0.128*"one"

Topic #3: 0.463*"islands" + 0.238*"island" + -0.184*"party" + -0.184*"damages" + -0.182*"contract" + 0.163*"antarctica" + 0.146*"ozone" + 0.138*"many" + 0.134*"pacific" + 0.121*"world"

Topic #4: 

Topic #5: 

Topic #6: 

Topic #7: 


In [110]:
coherence3 = CoherenceModel(model=lsi_model,
                           texts=preprocessed_docs, dictionary=dictionary, coherence='c_v')
print('Coherence score:', coherence3.get_coherence())

Coherence score: 0.5920302919899063
